# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)
sum= 0
for sqrt_value in generator:
    sum+=sqrt_value
    print(sqrt_value)
print (sum)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989
32.854555867161245


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [2]:

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [4]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX

In [20]:
# to do: homework :)
import dlt
pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')



# run with merge write disposition.
# This is so scaffolding is created for the next example,
# where we look at merging data

info = pipeline.run(people_1,table_name="p1",write_disposition="replace")
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.18 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////Users/mahmoud/learning/data_engineering_boot_camp/github/datacamp2024/data-ingestion-workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707780556.762403 is LOADED and contains no failed jobs


In [21]:
import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n http_download table below:")

rides = conn.sql("SELECT sum(age) FROM p1").df()
display(rides)


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ p1                  │
│ pe                  │
│ pe1                 │
│ pe2                 │
│ pe3                 │
└─────────────────────┘




 http_download table below:


,sum(age)
0,140.0


In [22]:
info = pipeline.run(people_2,table_name="p1",write_disposition="append")
print(info)
rides = conn.sql("SELECT sum(age) FROM p1").df()
display(rides)


Pipeline dlt_ipykernel_launcher load step completed in 0.16 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////Users/mahmoud/learning/data_engineering_boot_camp/github/datacamp2024/data-ingestion-workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707780573.155989 is LOADED and contains no failed jobs


,sum(age)
0,353.0


In [23]:
info =pipeline.run(people_1,table_name="p2", write_disposition="append", primary_key='ID')
print(info)
rides = conn.sql("SELECT sum(age) FROM p2").df()
display(rides)
data =conn.sql("select * from p2").df()
display(data)



Pipeline dlt_ipykernel_launcher load step completed in 0.18 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////Users/mahmoud/learning/data_engineering_boot_camp/github/datacamp2024/data-ingestion-workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707780599.2807791 is LOADED and contains no failed jobs


,sum(age)
0,140.0


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1707780599.2807791,VakLETGgbXmHbQ
1,2,Person_2,27,City_A,1707780599.2807791,CqCuSW9+wTzfXg
2,3,Person_3,28,City_A,1707780599.2807791,cCgpGed2corE9g
3,4,Person_4,29,City_A,1707780599.2807791,GNcZ/BzJXvVrYA
4,5,Person_5,30,City_A,1707780599.2807791,lKlHdYVUaj8YuQ


In [24]:
info=pipeline.run(people_2,table_name="p2",write_disposition="merge",primary_key="ID")
print(info)
rides = conn.sql("SELECT sum(age) FROM p2").df()
display(rides)
data =conn.sql("select * from p2").df()
display(data)

Pipeline dlt_ipykernel_launcher load step completed in 0.32 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////Users/mahmoud/learning/data_engineering_boot_camp/github/datacamp2024/data-ingestion-workshop/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707780608.4581819 is LOADED and contains no failed jobs


,sum(age)
0,266.0


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707780599.2807791,VakLETGgbXmHbQ,None
1,2,Person_2,27,City_A,1707780599.2807791,CqCuSW9+wTzfXg,None
2,3,Person_3,33,City_B,1707780608.4581819,9q8p3kbyktN0Dw,Job_3
3,6,Person_6,36,City_B,1707780608.4581819,NTCbCan2pmnVNQ,Job_6
4,5,Person_5,35,City_B,1707780608.4581819,TdJENDEcnmoN9A,Job_5
5,4,Person_4,34,City_B,1707780608.4581819,D55Hj3gKki/kuQ,Job_4
6,7,Person_7,37,City_B,1707780608.4581819,fFmQLTU7hdpFLQ,Job_7
7,8,Person_8,38,City_B,1707780608.4581819,8sn4eLMN4do8Fw,Job_8
